### Installation

In [8]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [20]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [11]:
from datasets import load_dataset
ComponentSoft_datasets = [
    load_dataset("ComponentSoft/k8s-kubectl", split = "train"),
    load_dataset("ComponentSoft/k8s-kubectl-35k", split = "train"),
    load_dataset("ComponentSoft/k8s-kubectl-cot-20k", split = "train")
]

for dataset in ComponentSoft_datasets:
    print(dataset.column_names)

instruction_datasets = [
    load_dataset("sozercan/k8s-instructions", split = "train"),
    load_dataset("dereklck/kubernetes_cli_dataset_20k", split = "train"),
    load_dataset("dereklck/kubernetes_operator_8b_2k", split = "train"),
    load_dataset("K8sAIOps/kubernetes_operator_dataset_1k", split = "train"),
    load_dataset("K8sAIOps/kubernetes_operator_3b_1.5k", split = "train"),
]

for dataset in instruction_datasets:
    print(dataset.column_names)

debug_datasets = [
    load_dataset("eliashasnat/k8s-qa", split = "train"),
    load_dataset("mcipriano/stackoverflow-kubernetes-questions", split = "train"),
    load_dataset("chenhunghan/k8s-debug-assistant-synthetic-data", split = "train"),
]

for dataset in debug_datasets:
    print(dataset.column_names)

# create yaml dataset HelloBoieeee/kubernetes_config, substratusai/k8s-instructions

['objective', 'command_name', 'command', 'description', 'syntax', 'flags', 'question', 'chain_of_thought']
['objective', 'command_name', 'command', 'description', 'syntax', 'flags', 'question', 'chain_of_thought']
['objective', 'command_name', 'command', 'description', 'syntax', 'flags', 'question', 'chain_of_thought']


Repo card metadata block was not found. Setting CardData to empty.


['instruction', 'output']
['instruction', 'type', 'output']
['instruction', 'type', 'output']
['instruction', 'type', 'output']
['instruction', 'type', 'output']
['Question', 'QuestionAuthor', 'Answer', 'AnswerAuthor']
['Question', 'QuestionAuthor', 'Answer', 'AnswerAuthor']
['prompt', 'completion', 'system_prompt']


In [12]:
from unsloth import to_sharegpt
from unsloth import standardize_sharegpt

def prepare_ComponentSoft(dataset):
    dataset = to_sharegpt(
        dataset,
        merged_prompt="Objective: {objective}\nCommand Name: {command_name}\nDescription: {description}\nSyntax: {syntax}[[\nChain of thought: {chain_of_thought}]] [[\nUser question: {question}]]",
        output_column_name="command",
        conversation_extension=3,  # Select more to handle longer conversations
    )
    return standardize_sharegpt(dataset)

def prepare_instruction(dataset):
    dataset = to_sharegpt(
        dataset,
        merged_prompt="{instruction}",
        output_column_name="output",
        conversation_extension=3,  # Select more to handle longer conversations
    )
    return standardize_sharegpt(dataset)


In [13]:
datasets_ready = [prepare_ComponentSoft(dataset) for dataset in ComponentSoft_datasets]
datasets_ready += [prepare_instruction(dataset) for dataset in instruction_datasets]

In [14]:
from datasets import load_dataset, concatenate_datasets
merged_dataset = concatenate_datasets(datasets_ready)

# Shuffle to avoid training biases
merged_dataset = merged_dataset.shuffle(seed=42)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [23]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = merged_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_expert",
        report_to = "none", # Use this for WandB etc
    ),
)

Applying chat template to train dataset (num_proc=2): 100%|██████████████████████████████████████████████████████████████████████| 114072/114072 [00:05<00:00, 19311.94 examples/s]
Truncating train dataset (num_proc=2): 100%|██████████████████████████████████████████████████████████████████████████████████████| 114072/114072 [00:21<00:00, 5320.79 examples/s]


In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 114,072 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.606100
2,0.640200
3,0.758500
4,0.739900
5,0.797000
6,0.741900
7,0.738800
8,0.644100
9,0.722100
10,0.565200


We can see the System and Instruction prompts are successfully masked!

In [25]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.988 GB.
4.85 GB of memory reserved.


In [29]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

75.7147 seconds used for training.
1.26 minutes used for training.
Peak reserved memory = 4.85 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 20.218 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [30]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": 'Attach to the first pod in a replica set named "node" and view its output'},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

kubectl attach replica-set/node:1 -c=my-container<|eot_id|>


In [52]:
model.save_pretrained("lora_k8s_operator")  # Local saving
tokenizer.save_pretrained("lora_k8s_operator")

('lora_k8s_operator/tokenizer_config.json',
 'lora_k8s_operator/special_tokens_map.json',
 'lora_k8s_operator/tokenizer.json')

In [ ]:
import os

token = os.getenv("HF_TOKEN")

In [37]:
model.push_to_hub("niradler/k8s_operator", token = token) # Online saving
tokenizer.push_to_hub("niradler/k8s_operator", token = token) # Online saving

adapter_model.safetensors: 100%|██████████████████████████████████████████████████████████████████████████████████| 97.3M/97.3M [00:19<00:00, 4.91MB/s]


Saved model to https://huggingface.co/niradler/k8s_operator


tokenizer.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 17.2M/17.2M [00:03<00:00, 5.28MB/s]


In [39]:
    model.push_to_hub_gguf(
        "niradler/k8s_operator", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = token, # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 14.94 out of 31.23 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 41.25it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0', 'q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...


RuntimeError: Unsloth: The file ('llama.cpp/llama-quantize' or 'llama.cpp/llama-quantize.exe' if you are on Windows WSL) or 'llama.cpp/quantize' does not exist.
But we expect this file to exist! Maybe the llama.cpp developers changed the name or check extension of the llama-quantize file.

In [41]:
model.save_pretrained_gguf("k8s_operator", tokenizer,)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 14.53 out of 31.23 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:01<00:00, 14.94it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...


RuntimeError: Unsloth: The file ('llama.cpp/llama-quantize' or 'llama.cpp/llama-quantize.exe' if you are on Windows WSL) or 'llama.cpp/quantize' does not exist.
But we expect this file to exist! Maybe the llama.cpp developers changed the name or check extension of the llama-quantize file.

In [51]:
model.save_pretrained_gguf("k8s_operator_q4_k_m", tokenizer, quantization_method = "q4_k_m")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 14.67 out of 31.23 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 47.81it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...


RuntimeError: Unsloth: The file ('llama.cpp/llama-quantize' or 'llama.cpp/llama-quantize.exe' if you are on Windows WSL) or 'llama.cpp/quantize' does not exist.
But we expect this file to exist! Maybe the llama.cpp developers changed the name or check extension of the llama-quantize file.